In [ ]:
# HYPER PARAM
PROJECT_NAME = "RobocupTrajectoryPrediction"
MODEL_NAME = "LitBiLSTM_2"
GROUP_NAME = "LitBiLSTM"
BATCH_SIZE = 512
EPOCHS = 200
HIDDEN_DIM = 256
NUM_LAYER = 4
LEARNING_RATE = 0.001

In [ ]:
import joblib
import os

import visualizer

from tqdm.notebook import tqdm
import datasets
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torchmetrics
import pytorch_lightning as pl

In [ ]:
torch.set_float32_matmul_precision("high")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
dataset = datasets.load_dataset(
    "ReonOhashi/RobocupTrajectoryPrediction_8team",
    revision="ab9aaf0feeb14402e112fb1bd94d26dd5d0ba507",
)
dataset = dataset["train"]
train_raw, test_raw = dataset.train_test_split(test_size=0.2, seed=42).values()
len(train_raw), len(test_raw)

In [ ]:
from typing import Union


def swap_rl(df):
    """左右のチーム情報を交換"""
    df = df.copy()
    df["l_name"], df["r_name"] = df["r_name"], df["l_name"]

    l_cols = [f"l{i}_x" for i in range(1, 12)] + [f"l{i}_y" for i in range(1, 12)]
    r_cols = [f"r{i}_x" for i in range(1, 12)] + [f"r{i}_y" for i in range(1, 12)]

    df[l_cols], df[r_cols] = df[r_cols].values, df[l_cols].values

    # x座標を反転
    df["b_x"] *= -1
    for i in range(1, 12):
        df[f"l{i}_x"] *= -1
        df[f"r{i}_x"] *= -1

    return df


def to_df(arrs):
    feature_columns = dataset.column_names
    columns = [col for col in feature_columns if col != "goal_type"]
    return [pd.DataFrame(arr, columns=columns).assign(goal_type=None) for arr in arrs]


def to_np(dataset: datasets.arrow_dataset.Dataset):
    # データをpd.DataFrameに変換
    # goal_typeがgoal_lの場合はそのまま、goal_rの場合は左右を入れ替える
    # goal_type列はその後削除
    a = [pd.DataFrame(i) for i in tqdm(dataset, leave=True)]
    a = [
        i if i["goal_type"].iloc[0] in ["goal_l", None] else swap_rl(i)
        for i in tqdm(a, leave=True)
    ]
    a = [i.drop(columns=["goal_type"]) for i in tqdm(a, leave=True)]
    return np.stack(a)


# Save to cache
# train, test = to_np(train_raw), to_np(test_raw)
# os.makedirs("/.cache/ohashi", exist_ok=True)
# joblib.dump((train, test), "/.cache/ohashi/train_test.pkl")
train, test = joblib.load("/.cache/ohashi/train_test.pkl")
train: np.ndarray
test: np.ndarray

print(train.shape, test.shape)

In [ ]:
# remove nan and inf
print(train.shape, test.shape)  # (5708, 50, 49) (1424, 50, 49)

train = train[~np.isnan(train).any(axis=(1, 2))]
test = test[~np.isnan(test).any(axis=(1, 2))]

print(train.shape, test.shape)

In [ ]:
X_train, y_train = train[:, :20, :], train[:, 20:, :]
X_test, y_test = test[:, :20, :], test[:, 20:, :]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
class EuclideanDistance(torchmetrics.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_state("sum", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("count", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds, target):
        indices = torch.arange(2, 26, 2)  # [2, 4, 6, ..., 24]
        final_preds = preds[:, -1, :]  # (b, 49)
        final_target = target[:, -1, :]  # (b, 49)

        errors = torch.sqrt(
            (final_preds[:, indices] - final_target[:, indices]) ** 2
            + (final_preds[:, indices + 1] - final_target[:, indices + 1]) ** 2
        )
        self.sum += torch.sum(errors)
        self.count += errors.size(0)

    def compute(self):
        return self.sum / self.count

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# TensorDataset と DataLoader の作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=16
)
val_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=16
)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger


class LitBiLSTM(pl.LightningModule):
    def __init__(
        self, input_dim, hidden_dim, output_dim, num_layers, seq_length, lr=0.001
    ):
        super().__init__()
        self.save_hyperparameters()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
        )
        self.fc = nn.Linear(hidden_dim * 2, output_dim * seq_length)
        self.criterion = nn.MSELoss()
        self.euclidean_distance = EuclideanDistance()
        self.train_losses = []
        self.val_losses = []

    def forward(self, x):
        out, _ = self.lstm(x)
        last_out = out[:, -1, :]
        output = self.fc(last_out)
        return output.view(-1, 30, self.hparams.output_dim)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        self.log(
            "train_euclidean_distance",
            self.euclidean_distance(y_hat, y),
            on_step=False,
            on_epoch=True,
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log(
            "val_euclidean_distance",
            self.euclidean_distance(y_hat, y),
            on_step=False,
            on_epoch=True,
        )
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer


# チェックポイントのコールバックを定義（val_euclidean_distanceが改善したときに保存）
checkpoint_callback = ModelCheckpoint(
    monitor="val_euclidean_distance",
    dirpath=f"checkpoints/{MODEL_NAME}",
    filename="bilstm-{epoch:02d}-{val_euclidean_distance:.4f}",
    save_top_k=3,
    mode="min",
)

wandb_logger = WandbLogger(
    project=PROJECT_NAME,
    log_model=True,
    save_code=True,
    save_dir="logs/",
    name=MODEL_NAME,
    group=GROUP_NAME,
)


trainer = pl.Trainer(
    max_epochs=EPOCHS,
    accelerator="gpu",
    devices=1,
    callbacks=[checkpoint_callback],
    logger=wandb_logger,
)


X_train.shape  # b, 20, 49
y_train.shape  # b, 30, 49

input_dim = X_train.shape[2]
hidden_dim = HIDDEN_DIM
output_dim = y_train.shape[2]
seq_length = y_train.shape[1]
num_layers = NUM_LAYER
learning_rate = LEARNING_RATE

In [ ]:
model = LitBiLSTM(
    input_dim, hidden_dim, output_dim, num_layers, seq_length, lr=learning_rate
)
trainer.fit(model, train_loader, val_loader)

In [ ]:
import time

best_model_path = checkpoint_callback.best_model_path
model = LitBiLSTM.load_from_checkpoint(best_model_path)
file_name = f"{MODEL_NAME}_{time.strftime('%Y%m%d%H%M%S')}.pth"

os.makedirs(f"models/{MODEL_NAME}", exist_ok=True)
torch.save(model.state_dict(), f"models/{MODEL_NAME}/{file_name}")

In [ ]:
import visualizer
from importlib import reload

reload(visualizer)
import visualizer


def visualize_test(X, y, index=0):

    X_test, y_test = X[index, :, :], y[index, :, :]
    X_test = np.array([X_test])

    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

    model.to(device)

    with torch.no_grad():
        pred = model.forward(X_test)

    pred = pred[0]
    X_test = X_test[0]

    X_test = X_test.cpu().numpy()
    pred = pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

    pred = np.concatenate([X_test, pred], axis=0)
    y_test = np.concatenate([X_test, y_test], axis=0)

    return visualizer.visualizer_np2(y_test, pred)


import pathlib

video_filepaths = []
for i in range(10):
    anim = visualize_test(X_test, y_test, index=i)
    video_filename = f"{MODEL_NAME}_{i}.gif"
    video_dir = pathlib.Path("videos")
    video_dir.mkdir(exist_ok=True)
    anim.save(video_dir / video_filename, writer="pillow", fps=10)
    video_filepaths.append(video_dir / video_filename)

wandb_logger.log_video("example", [str(i) for i in video_filepaths])

In [ ]:
euclidean_distance = EuclideanDistance().to(device)

error = 0
model.eval()
for X, y in tqdm(val_loader):
    X, y = X.to(device), y.to(device)
    with torch.no_grad():
        pred = model.forward(X)
    error += euclidean_distance(pred, y)
print(error / len(val_loader))